# Лабораторная работа №5. Классификатор

Используем написанный ранее автоэнокдер

In [9]:
import torch
import torch.nn as nn
import torchvision

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU())
        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid())
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(-1, 1, 28, 28)
        return x



In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(16, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = x.view(-1, 16)
        x = self.fc1(x)
        x = self.softmax(x)
        return x

autoencoder = Autoencoder()
classifier = Classifier()

criterion = nn.MSELoss()
autoencoder_optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

num_epochs = 10

Тренируем автоэнкодер

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        img = img.to(torch.device('cpu'))
        output = autoencoder(img)
        loss = criterion(output, img)
        autoencoder_optimizer.zero_grad()
        loss.backward()
        autoencoder_optimizer.step()
    print('Autoencoder - Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Autoencoder - Epoch [1/10], Loss: 0.0400
Autoencoder - Epoch [2/10], Loss: 0.0297
Autoencoder - Epoch [3/10], Loss: 0.0267
Autoencoder - Epoch [4/10], Loss: 0.0249
Autoencoder - Epoch [5/10], Loss: 0.0227
Autoencoder - Epoch [6/10], Loss: 0.0222
Autoencoder - Epoch [7/10], Loss: 0.0215
Autoencoder - Epoch [8/10], Loss: 0.0232
Autoencoder - Epoch [9/10], Loss: 0.0202
Autoencoder - Epoch [10/10], Loss: 0.0208


Используем данные энкодера и тренируем классификатор. Тренированный энкодер выделил основные признаки и понизил размерность исходных данных.

In [12]:
num_epochs = 10
for epoch in range(num_epochs):
    for data in train_loader:
        img, labels = data
        img = img.to(torch.device('cpu'))
        labels = labels.to(torch.device('cpu'))
        encoded = autoencoder.encoder(img.view(-1, 28*28))
        output = classifier(encoded)
        loss = criterion(output, torch.nn.functional.one_hot(labels, num_classes=10).float())
        classifier_optimizer.zero_grad()
        loss.backward()
        classifier_optimizer.step()
    print('Classifier - Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

total_mse = 0.0
total_samples = 0
with torch.no_grad():
    for data in test_loader:
        img, labels = data
        img = img.to(torch.device('cpu'))
        labels = labels.to(torch.device('cpu'))
        encoded = autoencoder.encoder(img.view(-1, 28*28))
        output = classifier(encoded)
        mse = criterion(output, torch.nn.functional.one_hot(labels, num_classes=10).float())
        total_mse += mse.item() * img.shape[0]
        total_samples += img.shape[0]
print('MSE: {:.4f}'.format(total_mse / total_samples))


Classifier - Epoch [1/10], Loss: 0.0477
Classifier - Epoch [2/10], Loss: 0.0452
Classifier - Epoch [3/10], Loss: 0.0374
Classifier - Epoch [4/10], Loss: 0.0439
Classifier - Epoch [5/10], Loss: 0.0245
Classifier - Epoch [6/10], Loss: 0.0280
Classifier - Epoch [7/10], Loss: 0.0331
Classifier - Epoch [8/10], Loss: 0.0189
Classifier - Epoch [9/10], Loss: 0.0216
Classifier - Epoch [10/10], Loss: 0.0189
MSE: 0.0203


Найдём точность классификатора. Сравним известные метки из датасета и полученные с помощью классификатора.

In [20]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        img, labels = data
        img = img.to(torch.device('cpu'))
        labels = labels.to(torch.device('cpu'))
        encoded = autoencoder.encoder(img.view(-1, 28*28))
        output = classifier(encoded)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy}")

Accuracy: 0.8670


Вывод: точность получилось высокая, однако её можно увеличить, если добавить дополнительные слои в автоэнкодер и классификатор, увеличить количество проходов обучения автоэнкодера и классификатора.